COMP 472 - Mini Project 1 - Fall 2021
Due October 18th, 2021
Team Samurai: Adrien Kamran (40095393), Abdul Sirawan (?)

**TASK 2**

**STEP 2 - Load the dataset in Python.**

In [1]:
import pandas
dataset = pandas.read_csv("drug200.csv")
# Print the first 5 rows (header incl.) to see if this works as intended
print(dataset.head(5))

   Age Sex      BP Cholesterol  Na_to_K   Drug
0   23   F    HIGH        HIGH   25.355  drugY
1   47   M     LOW        HIGH   13.093  drugC
2   47   M     LOW        HIGH   10.114  drugC
3   28   F  NORMAL        HIGH    7.798  drugX
4   61   F     LOW        HIGH   18.043  drugY


**STEP 3 - Plot the distribution of the instances in each class, then store the graphic in a file called "drug-distribution.pdf".**

In [2]:
import matplotlib.pyplot as plt
# For each column in the dataset...
for column in dataset:
    # Create a histogram of the data in the column's rows
    plt.hist(dataset[column])
    plt.title = column
    # Save the figure as 'column_name.png'
    plt.savefig(fname="distributions/"+column)
    # Clear the plot's figure window to make room for the next one
    plt.clf()

# Assemble the PDF in an image editor to make the collage, save as PDF

<Figure size 432x288 with 0 Axes>

Preliminary observations:
- Age: largest group between 40 and 50 yrs, smallest between 50 and 55 yrs.
- Sex: nearly even split.
- BP: more high than low, more low than normal.
- Cholesterol: more high than normal.
- Na to K: largest group between 10 and 15, smallest between 38 and 38.
- Drug: more drug Y than anything else, less drug C than anything else.

**STEP 4 - For the dataset, convert ordinal features to ordered numerical, and nominal features to unordered numerical.**

In [3]:
# from pandas.api.types import CategoricalDtype

# Convert unordered nominal features
dataset = pandas.get_dummies(dataset, columns=['Sex'], prefix=['Sex'])
# Convert ordered nominal features to their numeric codes
dataset['BP'] = dataset['BP'].apply(lambda x: ['LOW', 'NORMAL', 'HIGH'].index(x))
dataset['Cholesterol'] = dataset['Cholesterol'].apply(lambda x: ['NORMAL', 'HIGH'].index(x))
# Convert target classes to arbitrary numeric codes
dataset['Drug'].replace(to_replace=['drugA', 'drugB', 'drugC', 'drugX', 'drugY'], value=[1, 2, 3, 4, 5], inplace=True)
# Swap columns due to get_dummies placing new columns at the end of the dataframe
cols = list(dataset.columns)
a, b = cols.index('Sex_M'), cols.index('Drug')
cols[a], cols[b] = cols[b], cols[a]
dataset = dataset[cols]
print(dataset.head(5))

# Create ordered categories for ordinal features
# bp_cat = CategoricalDtype(categories = ['LOW', 'NORMAL', 'HIGH'], ordered = True)
# chol_cat = CategoricalDtype(categories = ['NORMAL', 'HIGH'], ordered = True)
# Assign ordered categories to respective features
# dataset['BP'] = dataset['BP'].astype(bp_cat)
# dataset['Cholesterol'] = dataset['Cholesterol'].astype(chol_cat)
# Find columns with category data type
# cat_columns = dataset.select_dtypes(['category']).columns
# dataset[cat_columns] = dataset[cat_columns].apply(lambda x: x.cat.codes)
# drug_cat = CategoricalDtype(categories = ['drugA', 'drugB', 'drugC', 'drugX', 'drugY'], ordered = False)
# dataset['Drug'] = dataset['Drug'].astype(drug_cat)
# Create unordered categories for nominal features
# sex_cat = CategoricalDtype(categories = ['M', 'F'], ordered = False)
# Assign unordered categories to respective features
# dataset['Sex'] = dataset['Sex'].astype(sex_cat)

   Age  BP  Cholesterol  Na_to_K  Sex_M  Sex_F  Drug
0   23   2            1   25.355      0      1     5
1   47   0            1   13.093      1      0     3
2   47   0            1   10.114      1      0     3
3   28   1            1    7.798      0      1     4
4   61   0            1   18.043      0      1     5


**STEP 5 - Split the dataset**

In [4]:
import sklearn.model_selection as skl
# First, split dataset into features and targets
features = dataset.iloc[:, 0:6] # The first five columns are features
target = dataset.iloc[:, 6:] # The last column is  the target
print(features.head(5))
print(target.head(5))
# Assign training and testing data to respective arrays
features_train, features_test, target_train, target_test = skl.train_test_split(features, target)
# Save training and testing data to files
features_train.to_csv(path_or_buf = 'features_train.csv')
features_test.to_csv(path_or_buf = 'features_test.csv')
target_train.to_csv(path_or_buf = 'target_train.csv')
target_test.to_csv(path_or_buf = 'target_test.csv')

   Age  BP  Cholesterol  Na_to_K  Sex_M  Sex_F
0   23   2            1   25.355      0      1
1   47   0            1   13.093      1      0
2   47   0            1   10.114      1      0
3   28   1            1    7.798      0      1
4   61   0            1   18.043      0      1
   Drug
0     5
1     3
2     3
3     4
4     5


**STEP 6 - Run classifiers**

**STEP 6.a - NB**

In [5]:
import statistics
import numpy as np
from datetime import datetime
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
# Create the output file for classifier results
pf = open("drug-performance.txt", "w")
print("** FILE CREATED:", datetime.now(), "**\n", file=pf)
# Create the Gaussian Classifier
gnb = GaussianNB()
# Train the model
nb_acc_list = []
nb_macavgf1_list = []
nb_wtavgf1_list = []
nb_avg_acc = -1
nb_avg_macavgf1 = -1
nb_avg_wtavgf1 = -1
nb_std_acc = -1
nb_std_macavgf1 = -1
nb_std_wtavgf1 = -1
for x in range(10):
      gnb.fit(features_train, target_train.values.ravel())
      target_prediction = gnb.predict(features_test)
      nb_acc_list.append(metrics.accuracy_score(target_test, target_prediction))
      nb_macavgf1_list.append(metrics.f1_score(target_test, target_prediction, average="macro"))
      nb_wtavgf1_list.append(metrics.f1_score(target_test, target_prediction, average="weighted"))
# Display the results (from the last run)
print("a)\n*****************************************************************************\nGAUSSIAN NAIVE BAYES\n", file=pf)
print("b)\nConfusion Matrix:\n", metrics.confusion_matrix(target_test, target_prediction), file=pf)
print("c)\nPrecision:\n", metrics.confusion_matrix(target_test, target_prediction),
      "\nRecall:\n", metrics.recall_score(target_test, target_prediction, average=None),
      "\nF1-measure:\n", metrics.f1_score(target_test, target_prediction, average=None), file=pf)
print("d)\nAccuracy:\n", metrics.accuracy_score(target_test, target_prediction),
      "\nMacro-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="macro"),
      "\nWeighted-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="weighted"),"\n", file=pf)

**STEP 6.b - Base-DT**

In [6]:
from sklearn.tree import DecisionTreeClassifier
# Create the DT Classifier
dtc = DecisionTreeClassifier()
# Train the model
bdt_acc_list = []
bdt_macavgf1_list = []
bdt_wtavgf1_list = []
bdt_avg_acc = -1
bdt_avg_macavgf1 = -1
bdt_avg_wtavgf1 = -1
bdt_std_acc = -1
bdt_std_macavgf1 = -1
bdt_std_wtavgf1 = -1
for x in range(10):
      dtc.fit(features_train, target_train.values.ravel())
      target_prediction = dtc.predict(features_test)
      bdt_acc_list.append(metrics.accuracy_score(target_test, target_prediction))
      bdt_macavgf1_list.append(metrics.f1_score(target_test, target_prediction, average="macro"))
      bdt_wtavgf1_list.append(metrics.f1_score(target_test, target_prediction, average="weighted"))
# Display the results
print("a)\n*****************************************************************************\nBASE DECISION TREE\n", file=pf)
print("b)\nConfusion Matrix:\n", metrics.confusion_matrix(target_test, target_prediction), file=pf)
print("c)\nPrecision:\n", metrics.confusion_matrix(target_test, target_prediction),
      "\nRecall:\n", metrics.recall_score(target_test, target_prediction, average=None),
      "\nF1-measure:\n", metrics.f1_score(target_test, target_prediction, average=None), file=pf)
print("d)\nAccuracy:\n", metrics.accuracy_score(target_test, target_prediction),
      "\nMacro-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="macro"),
      "\nWeighted-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="weighted"),"\n", file=pf)

**STEP 6.c - Top-DT**

In [7]:
from sklearn.model_selection import GridSearchCV
# Establish hyper-parameters to be used in evaluation
tree_parameters = {
      'criterion':['entropy', 'gini'],
      'max_depth':np.arange(3, 10),
      'min_samples_split':[3, 4, 5]
}
# Create an instance of the GridSearchCV
gscv = GridSearchCV(DecisionTreeClassifier(), tree_parameters)
# Run the exhaustive grid search on hyper-parameters, given the test data
gscv.fit(features_train, target_train.values.ravel())
# Retrieve the best estimator (which uses the best parameters)
print(gscv.best_estimator_)
# Use the best estimator in the decision tree classifier
tdt_acc_list = []
tdt_macavgf1_list = []
tdt_wtavgf1_list = []
tdt_avg_acc = -1
tdt_avg_macavgf1 = -1
tdt_avg_wtavgf1 = -1
tdt_std_acc = -1
tdt_std_macavgf1 = -1
tdt_std_wtavgf1 = -1
for x in range(10):
      target_prediction = gscv.best_estimator_.predict(features_test)
      tdt_acc_list.append(metrics.accuracy_score(target_test, target_prediction))
      tdt_macavgf1_list.append(metrics.f1_score(target_test, target_prediction, average="macro"))
      tdt_wtavgf1_list.append(metrics.f1_score(target_test, target_prediction, average="weighted"))
# Display the results
print("a)\n*****************************************************************************\nTOP DECISION TREE\n"
      "Best hyper-parameters:", gscv.best_params_, "\n", file=pf)
print("b)\nConfusion Matrix:\n", metrics.confusion_matrix(target_test, target_prediction), file=pf)
print("c)\nPrecision:\n", metrics.confusion_matrix(target_test, target_prediction),
      "\nRecall:\n", metrics.recall_score(target_test, target_prediction, average=None),
      "\nF1-measure:\n", metrics.f1_score(target_test, target_prediction, average=None), file=pf)
print("d)\nAccuracy:\n", metrics.accuracy_score(target_test, target_prediction),
      "\nMacro-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="macro"),
      "\nWeighted-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="weighted"),"\n", file=pf)

DecisionTreeClassifier(criterion='entropy', max_depth=4, min_samples_split=3)


**STEP 6.d - PER**

In [8]:
from sklearn.linear_model import Perceptron
# Create the Perceptron classifier
pc = Perceptron()
# Train the model
per_acc_list = []
per_macavgf1_list = []
per_wtavgf1_list = []
per_avg_acc = -1
per_avg_macavgf1 = -1
per_avg_wtavgf1 = -1
per_std_acc = -1
per_std_macavgf1 = -1
per_std_wtavgf1 = -1
for x in range(10):
      pc.fit(features_train, target_train.values.ravel())
      target_prediction = pc.predict(features_test)
      per_acc_list.append(metrics.accuracy_score(target_test, target_prediction))
      per_macavgf1_list.append(metrics.f1_score(target_test, target_prediction, average="macro"))
      per_wtavgf1_list.append(metrics.f1_score(target_test, target_prediction, average="weighted"))
# Display the results
print("a)\n*****************************************************************************\nPERCEPTRON\n", file=pf)
print("b)\nConfusion Matrix:\n", metrics.confusion_matrix(target_test, target_prediction), file=pf)
print("c)\nPrecision:\n", metrics.confusion_matrix(target_test, target_prediction),
      "\nRecall:\n", metrics.recall_score(target_test, target_prediction, average=None),
      "\nF1-measure:\n", metrics.f1_score(target_test, target_prediction, average=None), file=pf)
print("d)\nAccuracy:\n", metrics.accuracy_score(target_test, target_prediction),
      "\nMacro-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="macro"),
      "\nWeighted-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="weighted"),"\n", file=pf)

**STEP 6.e - Base-MLP**

In [9]:
from sklearn.neural_network import MLPClassifier
# Create the MLP classifier with instructed parameters
mlpc = MLPClassifier(hidden_layer_sizes=100, activation="logistic", solver="sgd")
# Train the model
bmlp_acc_list = []
bmlp_macavgf1_list = []
bmlp_wtavgf1_list = []
bmlp_avg_acc = -1
bmlp_avg_macavgf1 = -1
bmlp_avg_wtavgf1 = -1
bmlp_std_acc = -1
bmlp_std_macavgf1 = -1
bmlp_std_wtavgf1 = -1
for x in range(10):
      mlpc.fit(features_train, target_train.values.ravel())
      target_prediction = mlpc.predict(features_test)
      bmlp_acc_list.append(metrics.accuracy_score(target_test, target_prediction))
      bmlp_macavgf1_list.append(metrics.f1_score(target_test, target_prediction, average="macro"))
      bmlp_wtavgf1_list.append(metrics.f1_score(target_test, target_prediction, average="weighted"))
# Display the results
print("a)\n*****************************************************************************\nBASE MULTI-LAYERED PERCEPTRON\n", file=pf)
print("b)\nConfusion Matrix:\n", metrics.confusion_matrix(target_test, target_prediction), file=pf)
print("c)\nPrecision:\n", metrics.confusion_matrix(target_test, target_prediction),
      "\nRecall:\n", metrics.recall_score(target_test, target_prediction, average=None),
      "\nF1-measure:\n", metrics.f1_score(target_test, target_prediction, average=None), file=pf)
print("d)\nAccuracy:\n", metrics.accuracy_score(target_test, target_prediction),
      "\nMacro-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="macro"),
      "\nWeighted-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="weighted"),"\n", file=pf)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multi

**STEP 6.e - Top-MLP**

In [10]:
# Establish hyper-parameters to be used in evaluation
mlpc_parameters = {
      'activation':['identity', 'logistic', 'tanh', 'relu'],
      'hidden_layer_sizes':[(30, 50), (10, 10, 10)],
      'solver':['adam', 'sgd']
}
# Create an instance of the GridSearchCV
gscv = GridSearchCV(MLPClassifier(), mlpc_parameters)
# Run the exhaustive grid search on hyper-parameters, given the test data
gscv.fit(features_train, target_train.values.ravel())
# Retrieve the best estimator (which uses the best parameters)
print(gscv.best_estimator_)
# Use the best estimator in the decision tree classifier
tmlp_acc_list = []
tmlp_macavgf1_list = []
tmlp_wtavgf1_list = []
tmlp_avg_acc = -1
tmlp_avg_macavgf1 = -1
tmlp_avg_wtavgf1 = -1
tmlp_std_acc = -1
tmlp_std_macavgf1 = -1
tmlp_std_wtavgf1 = -1
for x in range(10):
      target_prediction = gscv.best_estimator_.predict(features_test)
      tmlp_acc_list.append(metrics.accuracy_score(target_test, target_prediction))
      tmlp_macavgf1_list.append(metrics.f1_score(target_test, target_prediction, average="macro"))
      tmlp_wtavgf1_list.append(metrics.f1_score(target_test, target_prediction, average="weighted"))
# Display the results
print("a)\n*****************************************************************************\nTOP MULTI-LAYERED PERCEPTRON\n"
      "Best hyper-parameters:", gscv.best_params_, "\n", file=pf)
print("b)\nConfusion Matrix:\n", metrics.confusion_matrix(target_test, target_prediction), file=pf)
print("c)\nPrecision:\n", metrics.confusion_matrix(target_test, target_prediction),
      "\nRecall:\n", metrics.recall_score(target_test, target_prediction, average=None),
      "\nF1-measure:\n", metrics.f1_score(target_test, target_prediction, average=None), file=pf)
print("d)\nAccuracy:\n", metrics.accuracy_score(target_test, target_prediction),
      "\nMacro-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="macro"),
      "\nWeighted-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="weighted"),"\n", file=pf)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multi

MLPClassifier(activation='tanh', hidden_layer_sizes=(30, 50))


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


**STEP 7 - Append results**
(This step is actually done in each of the sub-steps in Step 6)

**STEP 8 - Append statistics**
(Statistics calculated in each of the sub-steps in Step 6)

In [11]:
print("\n*****************************************************************************\nSTATISTICS\n", file=pf)

nb_avg_acc = statistics.mean(nb_acc_list)
nb_avg_macavgf1 = statistics.mean(nb_macavgf1_list)
nb_avg_wtavgf1 = statistics.mean(nb_wtavgf1_list)
nb_std_acc = np.std(nb_acc_list)
nb_std_macavgf1 = np.std(nb_macavgf1_list)
nb_std_wtavgf1 = np.std(nb_wtavgf1_list)
print("\nNB STATS:"
      "\nAverage accuracy:", nb_avg_acc,
      "\nAverage macro-average f1:", nb_avg_macavgf1,
      "\nAverage weighted-average f1:", nb_avg_wtavgf1,
      "\nStandard deviation accuracy:", nb_std_acc,
      "\nStandard deviation macro-average f1:", nb_std_macavgf1,
      "\nStandard deviation weighted-average f1:", nb_std_macavgf1, "\n", file=pf)

bdt_avg_acc = statistics.mean(bdt_acc_list)
bdt_avg_macavgf1 = statistics.mean(bdt_macavgf1_list)
bdt_avg_wtavgf1 = statistics.mean(bdt_wtavgf1_list)
bdt_std_acc = np.std(bdt_acc_list)
bdt_std_macavgf1 = np.std(bdt_macavgf1_list)
bdt_std_wtavgf1 = np.std(bdt_wtavgf1_list)
print("\nBASE-DT STATS:"
      "\nAverage accuracy:", bdt_avg_acc,
      "\nAverage macro-average f1:", bdt_avg_macavgf1,
      "\nAverage weighted-average f1:", bdt_avg_wtavgf1,
      "\nStandard deviation accuracy:", bdt_std_acc,
      "\nStandard deviation macro-average f1:", bdt_std_macavgf1,
      "\nStandard deviation weighted-average f1:", bdt_std_macavgf1, "\n", file=pf)

tdt_avg_acc = statistics.mean(tdt_acc_list)
tdt_avg_macavgf1 = statistics.mean(tdt_macavgf1_list)
tdt_avg_wtavgf1 = statistics.mean(tdt_wtavgf1_list)
tdt_std_acc = np.std(tdt_acc_list)
tdt_std_macavgf1 = np.std(tdt_macavgf1_list)
tdt_std_wtavgf1 = np.std(tdt_wtavgf1_list)
print("\nTOP-DT STATS:"
      "\nAverage accuracy:", tdt_avg_acc,
      "\nAverage macro-average f1:", tdt_avg_macavgf1,
      "\nAverage weighted-average f1:", tdt_avg_wtavgf1,
      "\nStandard deviation accuracy:", tdt_std_acc,
      "\nStandard deviation macro-average f1:", tdt_std_macavgf1,
      "\nStandard deviation weighted-average f1:", tdt_std_macavgf1, "\n", file=pf)

per_avg_acc = statistics.mean(per_acc_list)
per_avg_macavgf1 = statistics.mean(per_macavgf1_list)
per_avg_wtavgf1 = statistics.mean(per_wtavgf1_list)
per_std_acc = np.std(per_acc_list)
per_std_macavgf1 = np.std(per_macavgf1_list)
per_std_wtavgf1 = np.std(per_wtavgf1_list)
print("\nPER STATS:"
      "\nAverage accuracy:", per_avg_acc,
      "\nAverage macro-average f1:", per_avg_macavgf1,
      "\nAverage weighted-average f1:", per_avg_wtavgf1,
      "\nStandard deviation accuracy:", per_std_acc,
      "\nStandard deviation macro-average f1:", per_std_macavgf1,
      "\nStandard deviation weighted-average f1:", per_std_macavgf1, "\n", file=pf)

bmlp_avg_acc = statistics.mean(bmlp_acc_list)
bmlp_avg_macavgf1 = statistics.mean(bmlp_macavgf1_list)
bmlp_avg_wtavgf1 = statistics.mean(bmlp_wtavgf1_list)
bmlp_std_acc = np.std(bmlp_acc_list)
bmlp_std_macavgf1 = np.std(bmlp_macavgf1_list)
bmlp_std_wtavgf1 = np.std(bmlp_wtavgf1_list)
print("\nBASE-MLP STATS:"
      "\nAverage accuracy:", bmlp_avg_acc,
      "\nAverage macro-average f1:", bmlp_avg_macavgf1,
      "\nAverage weighted-average f1:", bmlp_avg_wtavgf1,
      "\nStandard deviation accuracy:", bmlp_std_acc,
      "\nStandard deviation macro-average f1:", bmlp_std_macavgf1,
      "\nStandard deviation weighted-average f1:", bmlp_std_macavgf1, "\n", file=pf)

tmlp_avg_acc = statistics.mean(tmlp_acc_list)
tmlp_avg_macavgf1 = statistics.mean(tmlp_macavgf1_list)
tmlp_avg_wtavgf1 = statistics.mean(tmlp_wtavgf1_list)
tmlp_std_acc = np.std(tmlp_acc_list)
tmlp_std_macavgf1 = np.std(tmlp_macavgf1_list)
tmlp_std_wtavgf1 = np.std(tmlp_wtavgf1_list)
print("\nTOP-MLP STATS:"
      "\nAverage accuracy:", tmlp_avg_acc,
      "\nAverage macro-average f1:", tmlp_avg_macavgf1,
      "\nAverage weighted-average f1:", tmlp_avg_wtavgf1,
      "\nStandard deviation accuracy:", tmlp_std_acc,
      "\nStandard deviation macro-average f1:", tmlp_std_macavgf1,
      "\nStandard deviation weighted-average f1:", tmlp_std_macavgf1, "\n", file=pf)

pf.close()